In [12]:
import pandas as pd
import nltk
import numpy
from sklearn.feature_extraction.text import CountVectorizer
import re
import pickle
from random import sample 
data=pd.read_csv("imdb_master.csv", encoding="latin-1")

In [13]:
data=data.loc[data.label.apply(lambda x: x=="neg" or x=="pos"),:]
data['label']=data.label.apply(lambda x: 1 if x=="pos" else 0)
data=data.sample(5000, random_state=123)
data.reset_index(inplace=True)

In [14]:
index_test=sample(range(5000),300)
index_train=list(set(range(5000)).difference(set(index_test)))
init_train=sample(index_train, 100)

In [15]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),\.!?]", " ", string)  
    string = re.sub(r"[^A-Za-z]", " ", string)  # remove numbers
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"e\.g\.,", " ", string) 
    string = re.sub(r"a\.k\.a\.", " ", string) 
    string = re.sub(r"i\.e\.,", " ", string) 
    string = re.sub(r"i\.e\.", " ", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"\'", "", string) 
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string)
    string = re.sub(r"br", "", string)
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " ( ", string) 
    string = re.sub(r"\)", " ) ", string) 
    string = re.sub(r"\?", " ? ", string)
    string = re.sub(r"\.", " . ", string)  
    string = re.sub(r"\s{2,}", " ", string) 
    string = re.sub(r"u\.s\.", " us ", string)
    return string.strip().lower().split()

In [16]:
text_clean = [clean_str(l) for l in data['review']]


In [17]:
import itertools
from collections import Counter
#create list to construct vocabulary
#clean_text_all = train_clean+test_clean
word_clean_count = Counter(list(itertools.chain.from_iterable(text_clean)))

In [18]:
#create dict of unique words
v_clean = [x for x, y in word_clean_count.items() if y >=5]
v_clean_all = [x for x, y in word_clean_count.items()]

In [19]:
#create vocabulary and inverse vocabulary. Words with frequency >5
inv_vocabulary_clean ={}
inv_vocabulary_clean[0] = 'END'
inv_vocabulary_clean[1] = 'UNK'
inv_vocabulary_clean[2] = '<PAD/>'
vocabulary_clean ={}
vocabulary_clean['END'] = 0
vocabulary_clean['UNK'] = 1
vocabulary_clean['<PAD/>'] = 2
ix=3
for v in v_clean_all:
    if v in v_clean:
        vocabulary_clean[v] = ix
        inv_vocabulary_clean[ix] = v
        ix +=1

In [20]:
#function to convert list of words to list of id's
def convert_word_to_ix_clean(data):
    result = []
    for sent in data:
        temp = []
        for w in sent:
            if w in vocabulary_clean:
                temp.append(vocabulary_clean.get(w,1))
            else:
                temp.append(1)
        temp.append(0)
        result.append(temp)
    return result

In [21]:
#convert list of words to list of id's
clean_num = convert_word_to_ix_clean(text_clean)


In [23]:
train_x_clean_num, train_x_clean, label_L=[clean_num[x] for x in init_train], [text_clean[x] for x in init_train], [data.label.tolist()[x] for x in init_train]
test_x_clean_num, test_x_clean, label_test=[clean_num[x] for x in index_test], [text_clean[x] for x in index_test], [data.label.tolist()[x] for x in index_test]
max_len_x=max([len(x) for x in train_x_clean_num])
max_len_test=max([len(x) for x in test_x_clean_num])
max_len_unlab=max([len(x) for x in clean_num])

In [24]:
import pickle
with open('model/random_sampling/M0.p', 'wb') as f:
    pickle.dump([train_x_clean_num, train_x_clean, label_L, vocabulary_clean, inv_vocabulary_clean, max_len_x], f, protocol=2)  
with open('model/random_sampling/test.p', 'wb') as f:
    pickle.dump([test_x_clean_num, test_x_clean, label_test, max_len_test], f, protocol=2) 
with open('model/random_sampling/all_valid_reports.p', 'wb') as f:
    pickle.dump([clean_num, text_clean,max_len_unlab], f, protocol=2)

In [161]:
import gensim
model_clean = gensim.models.Word2Vec(text_clean, size=300, window=5, min_count=5, workers=4)
model_clean.wv.save_word2vec_format('model/random_sampling/CTword2vec_clean','model/random_sampling/CTvocab_clean')

In [25]:
def valid(report):
    if report not in set(index_test) and report not in set(init_train):
        return True
    else:
        return False
def group(report):
    if report in set(index_test):
        return 'test'
    elif report in set(init_train):
        return 'control'
data['note_clinician']=float('NaN')
data['valid']=[valid(x) for x in list(data.index)]
data['group']=[group(x) for x in list(data.index)]

In [26]:
#data.drop(['type', 'index','Unnamed: 0',"file"], axis=1, inplace=True)

data['tokens_num']=[str(x) for x in clean_num

In [27]:
data["Report#"]=list(data.index)

In [167]:
data.to_csv('database.csv', encoding="latin-1")

In [168]:
data.to_csv('model/random_sampling/database.csv', encoding="latin-1")

In [174]:
with open('model/kidney/train.p', 'wb') as f:
    pickle.dump([train_x_clean_num, train_x_clean, label_L, vocabulary_clean, inv_vocabulary_clean, max_len_x], f, protocol=2) 

In [2]:
t=pd.read_csv("database.csv", encoding="latin-1")

In [28]:
g=data.loc[1,"tokens_num"]

In [29]:
import ast
ast.literal_eval(g)

ValueError: malformed string

In [30]:
g

[3001,
 1357,
 9240,
 6615,
 3541,
 277,
 5870,
 12183,
 3014,
 7063,
 977,
 8566,
 4801,
 10467,
 11495,
 7672,
 10024,
 3984,
 2801,
 11197,
 8640,
 12184,
 9194,
 7191,
 496,
 3984,
 2037,
 5870,
 10467,
 6279,
 11602,
 9615,
 1357,
 9427,
 3984,
 6114,
 1357,
 4927,
 10467,
 11353,
 847,
 2497,
 9417,
 9746,
 9577,
 10326,
 4667,
 258,
 7380,
 1832,
 431,
 8566,
 9615,
 4667,
 657,
 3613,
 3984,
 977,
 4801,
 3797,
 4801,
 10467,
 7063,
 7189,
 9311,
 1,
 3113,
 2867,
 3984,
 9444,
 4801,
 10129,
 6615,
 7321,
 3984,
 10675,
 1,
 2936,
 4794,
 347,
 2691,
 3619,
 3984,
 5500,
 5870,
 3984,
 6263,
 5870,
 1,
 2936,
 10697,
 6591,
 6615,
 6997,
 6405,
 2867,
 3984,
 10235,
 3984,
 9922,
 1,
 5870,
 9175,
 6263,
 9925,
 4794,
 10467,
 7063,
 7285,
 10670,
 1,
 11495,
 10326,
 4794,
 10467,
 9907,
 1695,
 5870,
 5235,
 6615,
 3613,
 9690,
 3984,
 3393,
 4794,
 10467,
 11403,
 5375,
 431,
 7809,
 6615,
 277,
 5870,
 141,
 3984,
 1176,
 6142,
 7839,
 3892,
 10467,
 12213,
 7239,
 2675,
 

In [22]:
clean_num[1]

[3001,
 1357,
 9240,
 6615,
 3541,
 277,
 5870,
 12183,
 3014,
 7063,
 977,
 8566,
 4801,
 10467,
 11495,
 7672,
 10024,
 3984,
 2801,
 11197,
 8640,
 12184,
 9194,
 7191,
 496,
 3984,
 2037,
 5870,
 10467,
 6279,
 11602,
 9615,
 1357,
 9427,
 3984,
 6114,
 1357,
 4927,
 10467,
 11353,
 847,
 2497,
 9417,
 9746,
 9577,
 10326,
 4667,
 258,
 7380,
 1832,
 431,
 8566,
 9615,
 4667,
 657,
 3613,
 3984,
 977,
 4801,
 3797,
 4801,
 10467,
 7063,
 7189,
 9311,
 1,
 3113,
 2867,
 3984,
 9444,
 4801,
 10129,
 6615,
 7321,
 3984,
 10675,
 1,
 2936,
 4794,
 347,
 2691,
 3619,
 3984,
 5500,
 5870,
 3984,
 6263,
 5870,
 1,
 2936,
 10697,
 6591,
 6615,
 6997,
 6405,
 2867,
 3984,
 10235,
 3984,
 9922,
 1,
 5870,
 9175,
 6263,
 9925,
 4794,
 10467,
 7063,
 7285,
 10670,
 1,
 11495,
 10326,
 4794,
 10467,
 9907,
 1695,
 5870,
 5235,
 6615,
 3613,
 9690,
 3984,
 3393,
 4794,
 10467,
 11403,
 5375,
 431,
 7809,
 6615,
 277,
 5870,
 141,
 3984,
 1176,
 6142,
 7839,
 3892,
 10467,
 12213,
 7239,
 2675,
 